In [ ]:
#%pip install -e ../.
#%pip uninstall -y polynx 
#%pip install update polars

Found existing installation: polynx 0.1.0
Uninstalling polynx-0.1.0:
  Successfully uninstalled polynx-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys
sys.path.insert(0, "../src")
import polynx
import gc
from polynx import DataFrame, LazyFrame
import polars as pl
import polynx as plx
gc.collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


542

In [5]:
#import pandas as pd
df = plx.read_csv("../../../../Downloads/Chase6317_Activity_20250107.CSV",truncate_ragged_lines=True)


In [12]:
df.head()


Details,Posting Date,Description,Amount,Type,Balance,Check or Slip #
str,str,str,f64,str,str,i64
"""DEBIT""","""01/07/2025""","""ONLINE TRANSFER TO CHK ...9592…",-1500.0,"""ACCT_XFER""",""" """,null
"""DEBIT""","""01/06/2025""","""HOME DEPOT AUTO PYMT 72…",-59.62,"""ACH_DEBIT""","""3348.72""",null
"""DEBIT""","""01/06/2025""","""Zelle payment to HENGYAN 23197…",-215.0,"""QUICKPAY_DEBIT""","""3408.34""",null
"""CREDIT""","""01/06/2025""","""WIREDCOMMUTE LLC PARKING-CR …",325.0,"""ACH_CREDIT""","""3623.34""",null
"""DEBIT""","""01/03/2025""","""IIT CITIBANK CITIXFR II…",-250.0,"""ACH_DEBIT""","""3298.34""",null


In [ ]:
df.query("'2024-01-01' <=`Posting Date`.str.to_date(strict=False) <'2025-01-01' & Amount < 0").vcnt('Type')

Type,count
str,u32
"""ACH_DEBIT""",67
"""ACCT_XFER""",25
"""DEBIT_CARD""",10
"""ATM""",4
"""LOAN_PMT""",4
"""QUICKPAY_DEBIT""",4
"""CHASE_TO_PARTNERFI""",3


In [64]:
(
    df.drop(['Balance','Check or Slip #','Details'])
    .query("'2024-01-01' <=`Posting Date`.str.to_date(strict=False) <'2025-01-01' & Amount < 0 & Type !='ACCT_XFER'")
    .describe(group_keys ='Type',selected_columns='Amount')
    .round(0)
    #.group_by('Type')    
    #.agg(pl.all().sample(1))
    #.agg(pl.all().first())
)

Type,Amount_count,Amount_nulls,Amount_mean,Amount_std,Amount_min,Amount_25%,Amount_50%,Amount_75%,Amount_max
str,u32,u32,f64,f64,f64,f64,f64,f64,f64
"""CHASE_TO_PARTNERFI""",3,0,-903.0,548.0,-1220.0,-1220.0,-1220.0,-270.0,-270.0
"""ATM""",4,0,-390.0,411.0,-1000.0,-240.0,-220.0,-220.0,-100.0
"""LOAN_PMT""",4,0,-1030.0,633.0,-1834.0,-1000.0,-1000.0,-1000.0,-286.0
"""QUICKPAY_DEBIT""",4,0,-178.0,74.0,-215.0,-215.0,-215.0,-215.0,-68.0
"""DEBIT_CARD""",10,0,-1053.0,44.0,-1110.0,-1077.0,-1048.0,-1039.0,-959.0
"""ACH_DEBIT""",67,0,-1895.0,1974.0,-5100.0,-3800.0,-886.0,-62.0,-21.0


In [92]:
(
    df.drop(['Balance','Check or Slip #','Details'])
    #.query("'2024-01-01' <=`Posting Date`.str.to_date(strict=False) <'2025-01-01' & Amount < 0 & Type !='ACCT_XFER' & ~Description.str.contains('NSM|CITIZE|GODD|Guo|Brooklyn|138|ATM|HOMETOWN')")
    .query("'2024-01-01' <=`Posting Date`.str.to_date(strict=False) <'2025-01-01' & Amount < 0 & Type !='ACCT_XFER' & Description.str.contains('(?i)Brooklyn|138')")
    .asort('Amount')
    .to_pandas()
    #.eval("Amount.sum()")
    #.query("'2024-01-01' <=`Posting Date`.str.to_date(strict=False) <'2025-01-01' & Amount < 0 & Type !='ACCT_XFER'")
    #.asort('Amount')
    #.gb('Type', '(Amount.sum()/1e3).round(1)')
    #.asort('Amount')
    #.group_by('Type').agg(pl.all().sample(1).first()).asort('Amount')
    #.group_by('Type').agg(pl.all().first())
)

,Posting Date,Description,Amount,Type
0,10/01/2024,YSI*Brooklyn Point 332-4564450 NY 1...,-1109.57,DEBIT_CARD
1,09/03/2024,YSI*Brooklyn Point 332-4564450 NY 0...,-1102.27,DEBIT_CARD
2,12/02/2024,YSI*Brooklyn Point 332-4564450 NY 1...,-1077.48,DEBIT_CARD
3,05/01/2024,YSI*Brooklyn Point 332-4564450 NY 0...,-1075.61,DEBIT_CARD
4,11/01/2024,YSI*Brooklyn Point 332-4564450 NY 1...,-1059.03,DEBIT_CARD
5,07/01/2024,YSI*Brooklyn Point 332-4564450 NY 0...,-1047.72,DEBIT_CARD
6,01/03/2024,138 Willoughby A PAYMENT 071001732233095 WE...,-1040.98,ACH_DEBIT
7,08/01/2024,YSI*Brooklyn Point 332-4564450 NY 0...,-1040.76,DEBIT_CARD
8,03/11/2024,YSI*Brooklyn Point 332-4564450 NY 0...,-1038.60,DEBIT_CARD
9,02/02/2024,138 Willoughby A PAYMENT 071001733139515 WE...,-1019.92,ACH_DEBIT


In [71]:
df = pl.DataFrame({
    'A':[1, 2, 3, 4], 
    'B': ['abc', 'bc', 'aaa', None], 
    'C': ['2023-01-01','2021-01-01','2009-11-01','2000-11-11'],
    'E': [1.1, 2.1, 3.5, 0]}
).with_columns(pl.col('C').str.to_datetime(format='%Y-%m-%d',time_unit='ns').dt.date())

df_plx = DataFrame(df)

In [12]:
#dir(df_plx.lazy())
df_plx.lazy().wc("E = 1").collect()
#type(df_plx)

A,B,C,E
i64,str,date,i32
1,"""abc""",2023-01-01,1
2,"""bc""",2021-01-01,1
3,"""aaa""",2009-11-01,1
4,null,2000-11-11,1


In [10]:
df_plx.lazy().describe()

statistic,A,B,C,E
str,f64,str,str,f64
"""count""",4.0,"""3""","""4""",4.0
"""null_count""",0.0,"""1""","""0""",0.0
"""mean""",2.5,null,"""2013-09-03 18:00:00""",1.675
"""std""",1.290994,null,null,1.488568
"""min""",1.0,"""aaa""","""2000-11-11""",0.0
"""25%""",2.0,null,"""2009-11-01""",1.1
"""50%""",3.0,null,"""2021-01-01""",2.1
"""75%""",3.0,null,"""2021-01-01""",2.1
"""max""",4.0,"""bc""","""2023-01-01""",3.5


In [12]:
#plx.DataFrame()
df_plx.describe(group_keys='B', selected_columns=['A','E']).round()

B,A_count,A_nulls,A_mean,A_std,A_min,A_25%,A_50%,A_75%,A_max,E_count,E_nulls,E_mean,E_std,E_min,E_25%,E_50%,E_75%,E_max
str,u32,u32,f64,f64,i64,f64,f64,f64,i64,u32,u32,f64,f64,f64,f64,f64,f64,f64
"""abc""",1,0,1.0,null,1,1.0,1.0,1.0,1,1,0,1.1,null,1.1,1.1,1.1,1.1,1.1
"""bc""",1,0,2.0,null,2,2.0,2.0,2.0,2,1,0,2.1,null,2.1,2.1,2.1,2.1,2.1
null,1,0,4.0,null,4,4.0,4.0,4.0,4,1,0,0.0,null,0.0,0.0,0.0,0.0,0.0
"""aaa""",1,0,3.0,null,3,3.0,3.0,3.0,3,1,0,3.5,null,3.5,3.5,3.5,3.5,3.5


In [11]:
var5 ='2001-01-01'
var3 = '2020-01-01'
query_str =  "@var1 <= A < @var2 & C.dt.year >=2020 & B in @var4"
query_str = "B.str.contains('a|b')"
query_str = "C >'2010-01-01' & A > 1"
#query_str = " B in ['aaa','bc']"
#query_str = " A == 1"
#query_str = " A ** 2 - E > 1"
#query_str = " @var5 < C < @var3 "
#query_str = " B.str.ends_with('c')"
#query_str = " not (B.is_null() & A.is_not_null())"
#query_str = " A ** 2 - E > 1"
query_str = "A.sum().over('B') > 1"

df_plx.query(query_str)

#dir(df)


A,B,C,E
i64,str,date,f64
2,"""bc""",2021-01-01,2.1
3,"""aaa""",2009-11-01,3.5
4,null,2000-11-11,0.0
